# SentencePiece



* 논문 : https://arxiv.org/pdf/1808.06226.pdf
* 센텐스피스 깃허브 : https://github.com/google/sentencepiece
* BPE를 포함하여 기타 서브워드 토크나이징 알고리즘들을 내장되어 있는 라이브러리.
* 사전 토큰화 작업없이 단어 분리 토큰화를 수행 -> 언어에 종속되지 않는다.


## 1. IMDB 리뷰 토큰화하기

In [2]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", 
                           filename="IMDB_Reviews.csv")

train_df = pd.read_csv("IMDB_Reviews.csv")
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [4]:
# sentencepiece 입력으로 사용하기 위해 txt 파일로 저장.

with open('imdb_review.txt','w',encoding='utf8') as f :
    f.write("\n".join(train_df['review']))

In [5]:
# sentencepiec 로 단어 집합과 각 단어에 고유한 정수 부여

import sentencepiece as spm

spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

각 인자가 의미하는 바
* `input` : 학습 시킬 파일
* `model_prefix` : 만들어질 모델 이름
* `vocab_size` : 단어 집합의 크기
* `model_type` : 사용할 모델(unigram(default), bpe, char, word)
* `max_sentence_length` : 문장의 최대 길이
* `pad_id, pad_piece` : pad token id, 값
* `unk_id, unk_piece` : unknown token id, 값
* `bos_id, bos_piece` : begin of sentence token id, 값
* `eos_id, eos_piece` : end of sentence token id, 값
* `user_definded_symbols` : 사용자 정의 토큰

In [13]:
# vocab 생성이 완료되면 imdb.model, imdb.vocab 파일 생성.
# vocab 파일에서 학습된 서브워드 확인 가능
import csv

vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
178,▁by,-175
114,her,-111
2329,ges,-2326
2799,▁Em,-2796
387,▁ad,-384
4552,reck,-4549
983,blem,-980
151,▁sc,-148
4794,apl,-4791
3919,▁Jon,-3916


In [14]:
# model 파일을 로드하여 인코딩/디코딩 작업 가능

sp = spm.SentencePieceProcessor()
sp.load('imdb.model')

True

내장 함수
* `encode_as_pieces(sentence)` : 문장 입력하면 서브 워드 시퀀스로 변환
* `encode_as_ids(sentence)` : 문장 입력하면 정수 시퀀스로 변환.
* `GetPieceSize()` : 단어 집합의 크기 확인.
* `IdToPiece(n)` : 정수로부터 맵핑되는 서브 워드로 변환
* `PieceToId(word)` : 서브워드로부터 맵핑되는 정수로 변환
* `DecodeIds(seq)` : 정수 시퀀스로부터 문장으로 변환
* `DecodePieces(seq)` : 서브워드 시퀀스로부터 문장으로 변환
* `encode(sentence,out_type)` : `out_type`이 `str` == 서브워드 시퀀스 , `int` == 정수 시퀀스 

In [19]:
sent = "I didn't at all think of it this way."

In [20]:
print(sp.encode_as_pieces(sent))
print(sp.encode_as_ids(sent))

['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]


In [24]:
print("vocabulary size : ",sp.GetPieceSize())
print("430 to ",sp.IdToPiece(430))
print("_character to ",sp.PieceToId("▁character"))

vocabulary size :  5000
430 to  ▁character
_character to  430


In [25]:
print(sp.DecodeIds([41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]))
print(sp.DecodePieces(['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']))

I didn't at all think of it this way.
I didn't at all think of it this way.


In [26]:
print(sp.encode(sent,out_type=str))
print(sp.encode(sent,out_type=int))

['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]


## 2. 네이버 영화 리뷰 토큰화하기

In [27]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", 
                           filename="ratings.txt")

naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [32]:
# 네이버 영화 리뷰 데이터의 경우 NULL 값 존재

print(naver_df.isnull().sum())
naver_df.dropna(inplace=True)
print(naver_df.isnull().sum())

id          0
document    8
label       0
dtype: int64
id          0
document    0
label       0
dtype: int64


In [33]:
with open('naver_review.txt','w',encoding='utf8') as f :
    f.write("\n".join(naver_df['document']))

In [35]:
# imdb 와 동일하게 진행
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [36]:
vocab_list = pd.read_csv('naver.vocab',sep='\t',header=None,quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
3137,이션,-3134
1217,이었다,-1214
3678,훈,-3675
3057,re,-3054
3637,왔,-3634
4475,짙,-4472
4902,섀,-4899
1858,없네,-1855
3670,열,-3667
168,액션,-165


In [37]:
sp = spm.SentencePieceProcessor()
sp.load('naver.model')

True

In [38]:
sent = "진짜 최고의 영화입니다 ㅋㅋ"

print(sp.encode_as_ids(sent))
print(sp.encode_as_pieces(sent))

[54, 200, 821, 85]
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
